In [8]:
import re
def process_diff(diff_text):
    # Replace <nl> with \n
    diff_text = diff_text.replace('<nl>', '\n')
    diff_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', diff_text)
    diff_text = re.sub(r'\n\s+', r'\n', diff_text)
    diff_text = diff_text.replace('mmm', 'diff --git')
    diff_text = diff_text.replace('\nppp', '')
    return diff_text

In [9]:
languages = ["java", "cpp", "csharp", "python", "javascript"]
mcmd_base_path = "../../../MCMD"

In [25]:
train_msg_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/train.msg.txt" for lang in languages]
train_diff_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/train.diff.txt" for lang in languages]

In [26]:
from tqdm import tqdm
selected_messages = []
selected_diffs = []
selected_languages = []
# use tqdm to show progress bar
for msg_path, diff_path in zip(train_msg_path, train_diff_path):
    count = 0
    # collect 100000 messages and diffs
    with open(msg_path, 'r', encoding='UTF-8') as f:
        messages = f.readlines()
    with open(diff_path, 'r', encoding='UTF-8') as f:
        diffs = f.readlines()
    # Iterate over messages and diffs simultaneously
    total_count = 100000
    with tqdm(total=total_count, desc="Processing messages") as pbar:
        for message, diff in zip(messages, diffs):
            # Check if message length is at least 5 characters
            if len(message.strip()) >= 5 and len(process_diff(diff)) <= 3000:
                selected_messages.append(message.strip())
                selected_diffs.append(process_diff(diff))
                selected_languages.append(msg_path.split('/')[-3])
                count += 1
                pbar.update(1)  # 更新进度条
            if count >= total_count:
                break

In [27]:
import json
data = []

assert len(selected_messages) == len(selected_diffs) == len(selected_languages) == 500000
for msg, diff, language in zip(selected_messages, selected_diffs, selected_languages):
    item = {
        'msg': msg,
        'diff': diff,
        'language': language
    }
    data.append(item)

# Write the data to a JSON file
with open('../data/rag_data.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [28]:
len(selected_messages)